# PLAYGROUND for AgentGENius

In [7]:
import logfire
from dotenv import load_dotenv
from rich import print

from agentgenius import AgentDef, Task, TaskDef
from agentgenius.builtin_tools import get_datetime, get_location_by_ip, get_user_ip
from agentgenius.tasks import TaskList
from agentgenius.tools import ToolSet

import nest_asyncio


nest_asyncio.apply()
load_dotenv()

True

In [8]:
agent_def = AgentDef(
    model="openai:gpt-4o-mini",
    name="assistant",
    system_prompt="You are a helpful assistant.",
    params={"retries": 2},
)

In [9]:
task_def = TaskDef(
    name="get time", question="what time is it?", priority=1, agent=agent_def, toolset=ToolSet([get_datetime])
)

In [10]:
task = Task(task=task_def)
print(task.run_sync().data)

The current time is 22:40:11 on January 9, 2025.

In [42]:
from agentgenius.config import prompt_lib
from pydantic.dataclasses import dataclass


@dataclass
class ToolCode:
    name: str
    code: str


class CoderTask(TaskDef):
    def __init__(self, question=""):
        self.name = "coder"
        self.question = "write a python function give answer to my question: " + question
        self.priority = 1
        self.agent = AgentDef(
            model="openai:gpt-4o",
            name="coder",
            system_prompt=prompt_lib["tool_agent"],
            params={"retries": 2, "result_type": ToolCode},
        )
        self.toolset = ToolSet([])


coder = Task(CoderTask("what time is it?"))
print(coder.run_sync().data)

ToolCode(
    name='get_current_time',
    code='def get_current_time() -> str:\n    """\n    Returns the current local time as a string in the format 
\'HH:MM:SS\'.\n    """\n    from datetime import datetime\n    return datetime.now().strftime(\'%H:%M:%S\')'
)